In [1]:
import pandas as pd
import numpy as np
from Utilities_fantapalla import *
from joblib import Parallel, delayed
import progressbar
quotazioni = pd.read_csv ('Quotazioni_Fantacalcio_ridotte_portieri.csv')

In [38]:
portieri = np.array([2, 0, 0, 0])
resto = np.array([0, 7, 7, 4])
num_squadre = 8
teams, team_names = names(num_squadre)
N_cores = 8

In [3]:
rosa_portieri = genera_rose(portieri, num_squadre)

In [4]:
quot_dict_portieri = all_quot_dict(portieri, quotazioni, num_squadre)
quot_dict_resto = all_quot_dict(resto, quotazioni, num_squadre)

In [35]:
def spread(rosa, team_names, quot_dict):
    spread = np.float(np.max(assign_quot(rosa, quot_dict, team_names))-np.min(assign_quot(rosa, quot_dict, team_names)))
    return spread

In [36]:
def find_rosa(struttura_rosa, num_squadre, quotazioni, team_names, iterations = 100):
    rosa = genera_rose(struttura_rosa, num_squadre)
    quot_dict = all_quot_dict(struttura_rosa, quotazioni, num_squadre)
    rose_generate = Parallel(n_jobs = N_cores)(delayed(genera_rose)(struttura_rosa, num_squadre) for i in progressbar.progressbar(range(iterations)))
    all_spread = Parallel(n_jobs = N_cores)(delayed(spread)(rosa, team_names, quot_dict) for rosa in progressbar.progressbar(rose_generate))
    min_ind = np.argmin(all_spread)
    output = pd.DataFrame(data = rose_generate[min_ind], columns = team_names)
    return output

In [48]:
rosa_best_portieri = find_rosa(portieri, num_squadre, quotazioni, team_names, iterations = 25000000)

100% (25000000 of 25000000) |############| Elapsed Time: 0:20:50 Time:  0:20:50
100% (25000000 of 25000000) |############| Elapsed Time: 2:14:12 Time:  2:14:12


In [49]:
assign_quot(rosa_best_portieri, quot_dict_portieri, team_names)

,0
Team 1,15.0
Team 2,12.0
Team 3,16.0
Team 4,15.0
Team 5,16.0
Team 6,12.0
Team 7,16.0
Team 8,12.0


In [39]:
rosa_best_resto = find_rosa(resto, num_squadre, quotazioni, team_names, iterations = 25000000)

100% (25000000 of 25000000) |############| Elapsed Time: 0:22:27 Time:  0:22:27
100% (25000000 of 25000000) |############| Elapsed Time: 2:54:34 Time:  2:54:34


In [50]:
assign_quot(rosa_best_resto, quot_dict_resto, team_names)

,0
Team 1,218.0
Team 2,218.0
Team 3,220.0
Team 4,218.0
Team 5,219.0
Team 6,218.0
Team 7,220.0
Team 8,220.0


In [51]:
nomi_portieri = id_toName(portieri, quotazioni, rosa_best_portieri, num_squadre, team_names)
nomi_resto = id_toName(resto, quotazioni, rosa_best_resto, num_squadre, team_names)

In [53]:
struttura_parziale = np.array([2,7,7,4])
nomi_resto.index =  list(range(2,20))
rosa_parziale = pd.concat([nomi_portieri, nomi_resto], axis = 0)
#rosa_parziale.to_pickle('rose_parziali_best_portieri.pkl')
FC_colors(rosa_parziale, struttura_parziale)

Team 1,Team 2,Team 3,Team 4,Team 5,Team 6,Team 7,Team 8
MONTIPO',ROSSI F.,CONSIGLI,SEPE,PERIN,SIRIGU,SKORUPSKI,SPORTIELLO
MERET,MUSSO,CORDAZ,ZOET,OSPINA,CARNESECCHI,AUDERO,SILVESTRI
VOJVODA,GODIN,ALEX SANDRO,GLIK,IZZO,MAKSIMOVIC,ANSALDI,DIJKS
STRYGER LARSEN,BASTONI,BIRAGHI,IBANEZ,FERRARI G.,COLLEY,MANCINI,ROMAGNOLI
PEZZELLA GER.,KOLAROV,DE SILVESTRI,SPINAZZOLA,NUYTINCK,ROMERO,MUSACCHIO,DJIMSITI
RODRIGUEZ R.,DI LORENZO,CHIELLINI,BONUCCI,BANI,BRUNO ALVES,KUMBULLA,BRUNO PERES
RADU,GAGLIOLO,FARAONI,D'AMBROSIO,CUADRADO,LIROLA,SKRINIAR,HATEBOER
MARIO RUI,AUGELLO,DARMIAN,LUIZ FELIPE,KOULIBALY,CACERES,PALOMINO,CEPPITELLI
TOLOI,MANOLAS,KJAER,TOMIYASU,N'KOULOU,RRAHMANI,DANILO LAR.,YOUNG
ROG,LAZZARI,SCHONE,ZAJC,MILINKOVIC-SAVIC,BONAVENTURA,BARELLA,SORIANO


In [54]:
rosa_parzial_pickle = pd.read_pickle('rose_parziali_best_portieri.pkl')
FC_colors(rosa_parzial_pickle, struttura_parziale)

Team 1,Team 2,Team 3,Team 4,Team 5,Team 6,Team 7,Team 8
MONTIPO',ROSSI F.,CONSIGLI,SEPE,PERIN,SIRIGU,SKORUPSKI,SPORTIELLO
MERET,MUSSO,CORDAZ,ZOET,OSPINA,CARNESECCHI,AUDERO,SILVESTRI
VOJVODA,GODIN,ALEX SANDRO,GLIK,IZZO,MAKSIMOVIC,ANSALDI,DIJKS
STRYGER LARSEN,BASTONI,BIRAGHI,IBANEZ,FERRARI G.,COLLEY,MANCINI,ROMAGNOLI
PEZZELLA GER.,KOLAROV,DE SILVESTRI,SPINAZZOLA,NUYTINCK,ROMERO,MUSACCHIO,DJIMSITI
RODRIGUEZ R.,DI LORENZO,CHIELLINI,BONUCCI,BANI,BRUNO ALVES,KUMBULLA,BRUNO PERES
RADU,GAGLIOLO,FARAONI,D'AMBROSIO,CUADRADO,LIROLA,SKRINIAR,HATEBOER
MARIO RUI,AUGELLO,DARMIAN,LUIZ FELIPE,KOULIBALY,CACERES,PALOMINO,CEPPITELLI
TOLOI,MANOLAS,KJAER,TOMIYASU,N'KOULOU,RRAHMANI,DANILO LAR.,YOUNG
ROG,LAZZARI,SCHONE,ZAJC,MILINKOVIC-SAVIC,BONAVENTURA,BARELLA,SORIANO


In [10]:
import scipy as sp
import scipy.special

In [11]:
sp.special.binom(10,4)

210.0

In [12]:
tempP = 1
k = 2
N = 16
while k < N:
    tempP = tempP * sp.special.binom(N,k)
    N = N-k

tempD = 1
k = 7
N = 56
while k < N:
    tempD = tempD * sp.special.binom(N,k)
    N = N-k
    
tempC = tempD    

tempA = 1
k = 4
N = 32
while k < N:
    tempA = tempA * sp.special.binom(N,k)
    N = N-k

In [13]:
tempP*tempD*tempC*tempA

5.697841118500126e+125

In [14]:
tempP

81729648000.0